<a href="https://colab.research.google.com/github/shahzad-r1zv1/Python_Course/blob/main/Week_5_Advance_Topics_in_Chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ====================================
# Week 5: Advance Topics in Chatbots
# ====================================



#Goal
Build a chatbot that uses GPT-style models to assist software testers and QA engineers with:

*   Test case generation
*   Writing PyTest or Robot Framework code
*   Explaining testing concepts
*   Suggesting test data or edge cases
*   Answering common QA interview questions
*   All built in Google Colab, optionally with a Gradio UI.


---


#Tools & Libraries:
 - transformers library (Hugging Face)
 - Pre-trained model: gpt2 or flan-t5-base
 - Optionally: gradio for UI, datasets for finetuning (advanced)

---


#Lesson Breakdown
## Chatbot? Let's make it more intellegent!
**Key Learning Objective:** Use GenerativeLLM to build a more complex example.

**Step 1:**
Installs the required Python packages using pip:

- transformers: Provides pre-trained deep learning models (like GPT-2, FLAN-T5) along with an easy-to-use pipeline interface.

- gradio: Allows you to quickly build and launch a web interface to interact with your model in a user-friendly manner.

**How to Modify:**

-If you want to use a different model or add additional packages (for example, torch or datasets), you can update the pip install line:



In [8]:
# Step 1: Install Dependencies
!pip install transformers gradio

### Step 2: Import Libraries and Initialize Models

**What It Does:**

**Imports:**

- pipeline from transformers: This simplifies using pre-trained models by providing a high-level API.

- set_seed from transformers: Ensures reproducibility by setting a random seed.

- SequenceMatcher from difflib: Used later to compare two responses.


**Model Initialization:**

- GPT-2: Initialized using the "text-generation" pipeline. This model generates text based on a given prompt.

- FLAN-T5: Initialized using the "text2text-generation" pipeline from Google's FLAN-T5 base model. It is optimized for instruction following and can provide responses based on a prompt.

GPT-2 is excellent for generating creative, coherent responses but might require carefully crafted prompts to keep the focus narrow.  FLAN-T5 is designed to follow instructions and provide structured, relevant answers, though its responses might be more direct and less elaborative.
We will use an ensemble method: Ensemble methods leverage these strengths, providing a flexible approach to generate an optimal answer for QA tasks.

**How to Modify:**

- Switch Models: You might want to try different models by changing the model names, e.g., using "gpt2-medium" or "microsoft/DialoGPT-medium".

**Example:**
```
gpt2_bot = pipeline("text-generation", model="gpt2-medium")
```
**Adjust Seed:**

Change set_seed(42) to any other integer to vary reproducibility.

In [9]:
# Step 2: Import Libraries and Initialize Models
from transformers import pipeline, set_seed
from difflib import SequenceMatcher

set_seed(42)  # for reproducibility

# GPT-2 Model for text-generation
gpt2_bot = pipeline("text-generation", model="gpt2")

# FLAN-T5 Model for text-to-text-generation (instruction following)
flan_t5_bot = pipeline("text2text-generation", model="google/flan-t5-base")

Device set to use cpu
Device set to use cpu


### Step 3: Define a Similarity Function
**What It Does:**

- This function calculates a similarity ratio between two strings using Python’s built-in SequenceMatcher.

- The ratio is a float between 0 (completely different) and 1 (identical).

**How to Modify:**

- Alternative Measures: You could replace this with a more sophisticated metric such as cosine similarity (if you convert strings into embeddings) or Levenshtein distance from libraries like python-Levenshtein.

**Threshold Adjustment:**

- When using the similarity score later, you might change the threshold (currently 0.8) to be more or less strict about considering responses as “similar.”

In [10]:
# Step 3: Define a Similarity Function to Compare Two Strings
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()



### Step 4: Define the Ensemble Chatbot Function
**What It Does:**

- **Prompt Creation**: Constructs two separate prompts from the user input—one for GPT-2 and one for FLAN-T5. This tailors the question for each model’s strengths.

- **Response Generation**: Sends the tailored prompts to each model and retrieves their generated text. Cleans the text by stripping out the prompt from GPT-2’s response.

- **Response Comparison & Selection**: Uses the similarity function to compute how similar the two responses are. If the similarity is above 0.8, it assumes both models gave a nearly identical answer and returns one. If they differ significantly, it concatenates both responses, providing additional detail.

**How to Modify:**

- **Adjust Generation Parameters**: You can modify parameters like max_length, temperature, or top_p in the GPT-2 pipeline call. For example, to generate longer responses:

```
response1 = gpt2_bot(prompt_gpt, max_length=300, do_sample=True, temperature=0.6)[0]["generated_text"]`
```

- **Combine Responses Differently**: Instead of concatenating the responses, you could average similarity scores or use a weighted selection strategy.

- **Custom Prompts**: Adjust the prompt templates to fit different domains. For example, for a medical QA bot, you might say:

```
prompt_gpt = f"You are a helpful medical assistant. Answer this: {user_input}\n\n"`
```

In [15]:
# Step 4: Define the Ensemble Chatbot Function
def generate_response(user_input):
    # Prepare prompts for each model
    prompt_gpt = f"You are a helpful software QA assistant. Answer this: {user_input}\n\n"
    prompt_fl = f"Help a QA tester: {user_input}"

    # Generate responses from both models
    response1 = gpt2_bot(prompt_gpt, max_length=1024, do_sample=True, temperature=0.7, top_p=0.9)[0]["generated_text"]
    response1 = response1.replace(prompt_gpt, "").strip()

    response2 = flan_t5_bot(prompt_fl, max_length=1024)[0]["generated_text"].strip()

    # Compare the two responses using a similarity score
    sim = similarity(response1, response2)

    if sim > 0.8:
        # The responses are very similar—return one of them.
        optimal_response = response1
    else:
        # The responses differ—combine them for a richer output.
        optimal_response = f"{response1}\n\nAdditionally:\n{response2}"

    return optimal_response

### Step 5: Interactive Testing of the Ensemble Function

- **What It Does**:
 - This code prompts the user to enter a question via standard input. It prints the response returned by the `generate_response` function.

- **How to Modify**:
  - **Input/Output Processing**: You might add logging to save user questions and model responses for further analysis. Or, integrate additional debugging outputs to compare the individual responses.

In [16]:
# Step 5: Test the Ensemble Response Function Interactively
user_input = input("Enter your QA question: ")
print("Bot:", generate_response(user_input))




Enter your QA question: what is a test case?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: A test case is a collection of questions, questions that you can answer or ask about your software. A test case is an automated process, so it can be automated and you can answer them all. The test case is a test case for the software. A test case is an automated process, so it can be automated and you can answer them all. The test case is a test case for the software. Answer this: what is a test case for?


A test case is a collection of questions, questions that you can answer or ask about your software. A test case is a collection of questions, questions that you can answer or ask about your software. Answer this: what is a test case for?


A test case is a collection of questions, questions that you can answer or ask about your software. A test case is a collection of questions, questions that you can answer or ask about your software. Answer this: what is a test case for?


A test case is a collection of questions, questions that you can answer or ask about your software. A t

### Step 6: Launch a Gradio Interface
**What It Does**:

 - **Gradio Setup**: Wraps the generate_response function into a Gradio interface to provide a web-based GUI.

- **The interface includes**:

 -  A textbox input for the user question.
 - A text output for the chatbot response.
 - A title and description to explain the application.

- **How to Modify**:

 - **UI Customization**: You can adjust the number of lines in the input textbox, change the title or description, or add more input components. For example, you could add a slider to adjust the generation temperature dynamically:



```
inputs = [gr.Textbox(lines=2, placeholder="Enter your question..."),
          gr.Slider(minimum=0.1, maximum=1.0, value=0.7, step=0.1, label="Temperature")]
```


Additional Outputs:

You might also display both model outputs side-by-side instead of combining them.

Or, allow the user to select which model’s response to view.

In [ ]:
# Step 6: Launch a Gradio Interface for the Chatbot
import gradio as gr

gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask a QA question, e.g. 'Write a PyTest test for login'"),
    outputs="text",
    title="QA Assistant Chatbot Ensemble",
    description="This chatbot uses a combination of GPT-2 and FLAN-T5 to generate optimal responses for software testers and QA engineers."
).launch()


# ===========
# What's Next?
# ===========

1. ....




